# Derive models of drone

Do all imports.

In [2]:
import sympy as sym
import numpy as np

# Suppress the use of scientific notation when printing small numbers
np.set_printoptions(suppress=True)

## Dynamic model

Define physical parameters.

In [3]:
params = {
    'm': 0.5,
    'Jx': 0.0023,
    'Jy': 0.0023,
    'Jz': 0.0040,
    'l': 0.175,
    'g': 9.81,
}

Derive the equations of motion:

In [4]:
# components of position (meters)
p_x, p_y, p_z = sym.symbols('p_x, p_y, p_z')

# yaw, pitch, roll angles (radians)
psi, theta, phi = sym.symbols('psi, theta, phi')

# components of linear velocity (meters / second)
v_x, v_y, v_z = sym.symbols('v_x, v_y, v_z')
v_in_body = sym.Matrix([v_x, v_y, v_z])

# components of angular velocity (radians / second)
w_x, w_y, w_z = sym.symbols('w_x, w_y, w_z')
w_in_body = sym.Matrix([w_x, w_y, w_z])

# components of net rotor torque
tau_x, tau_y, tau_z = sym.symbols('tau_x, tau_y, tau_z')

# net rotor force
f_z = sym.symbols('f_z')

# parameters
m = sym.nsimplify(params['m'])
Jx = sym.nsimplify(params['Jx'])
Jy = sym.nsimplify(params['Jy'])
Jz = sym.nsimplify(params['Jz'])
l = sym.nsimplify(params['l'])
g = sym.nsimplify(params['g'])
J = sym.diag(Jx, Jy, Jz)

# rotation matrices
Rz = sym.Matrix([[sym.cos(psi), -sym.sin(psi), 0], [sym.sin(psi), sym.cos(psi), 0], [0, 0, 1]])
Ry = sym.Matrix([[sym.cos(theta), 0, sym.sin(theta)], [0, 1, 0], [-sym.sin(theta), 0, sym.cos(theta)]])
Rx = sym.Matrix([[1, 0, 0], [0, sym.cos(phi), -sym.sin(phi)], [0, sym.sin(phi), sym.cos(phi)]])
R_body_in_world = Rz @ Ry @ Rx

# angular velocity to angular rates
ex = sym.Matrix([[1], [0], [0]])
ey = sym.Matrix([[0], [1], [0]])
ez = sym.Matrix([[0], [0], [1]])
M = sym.simplify(sym.Matrix.hstack((Ry @ Rx).T @ ez, Rx.T @ ey, ex).inv(), full=True)

# applied forces
f_in_body = R_body_in_world.T @ sym.Matrix([[0], [0], [-m * g]]) + sym.Matrix([[0], [0], [f_z]])

# applied torques
tau_in_body = sym.Matrix([[tau_x], [tau_y], [tau_z]])

# equations of motion
f = sym.Matrix.vstack(
    R_body_in_world @ v_in_body,
    M @ w_in_body,
    (1 / m) * (f_in_body - w_in_body.cross(m * v_in_body)),
    J.inv() @ (tau_in_body - w_in_body.cross(J @ w_in_body)),
)

f = sym.simplify(f, full=True)

The equations of motion have this form:

$$\begin{bmatrix} \dot{p}_x \\ \dot{p}_y \\ \dot{p}_z \\ \dot{\psi} \\ \dot{\theta} \\ \dot{\phi} \\ \dot{v}_x \\ \dot{v}_y \\ \dot{v}_z \\ \dot{w}_x \\ \dot{w}_y \\ \dot{w}_z \end{bmatrix} = f\left(p_x, p_y, p_z, \psi, \theta, \phi, v_x, v_y, v_z, w_x, w_y, w_z, \tau_x, \tau_y, \tau_z, f_z \right)$$

Here is the function $f$:

In [5]:
f

Matrix([
[v_x*cos(psi)*cos(theta) + v_y*(sin(phi)*sin(theta)*cos(psi) - sin(psi)*cos(phi)) + v_z*(sin(phi)*sin(psi) + sin(theta)*cos(phi)*cos(psi))],
[v_x*sin(psi)*cos(theta) + v_y*(sin(phi)*sin(psi)*sin(theta) + cos(phi)*cos(psi)) - v_z*(sin(phi)*cos(psi) - sin(psi)*sin(theta)*cos(phi))],
[                                                                      -v_x*sin(theta) + v_y*sin(phi)*cos(theta) + v_z*cos(phi)*cos(theta)],
[                                                                                                 (w_y*sin(phi) + w_z*cos(phi))/cos(theta)],
[                                                                                                              w_y*cos(phi) - w_z*sin(phi)],
[                                                                                  w_x + w_y*sin(phi)*tan(theta) + w_z*cos(phi)*tan(theta)],
[                                                                                                   v_y*w_z - v_z*w_y + 981*sin(theta)/100],
[   

## Sensor model

Define the sensor model.

In [20]:
# Position of drone in world frame
p_in_world = sym.Matrix([p_x, p_y, p_z])

# Position of markers in body frame
a_in_body = sym.Matrix([l, 0, 0])  # <-- marker on front rotor
b_in_body = sym.Matrix([-l, 0, 0]) # <-- marker on rear rotor

# Position of markers in world frame
a_in_world = p_in_world + R_body_in_world @ a_in_body
b_in_world = p_in_world + R_body_in_world @ b_in_body

# Sensor model
g = sym.simplify(sym.Matrix.vstack(a_in_world, b_in_world))

The sensor model has this form:

$$o = g(p_x, p_y, p_z, \psi, \theta)$$

Here is the function $g$:

In [21]:
g

Matrix([
[p_x + 7*cos(psi)*cos(theta)/40],
[p_y + 7*sin(psi)*cos(theta)/40],
[         p_z - 7*sin(theta)/40],
[p_x + 7*cos(psi)*cos(theta)/40],
[p_y + 7*sin(psi)*cos(theta)/40],
[         p_z - 7*sin(theta)/40]])

In [34]:
p_xe = 0
p_ye = 0 
p_ze = 0
psie = 0 #np.pi/2
thetae = 0
phie = 0 #np.pi/2
v_xe = 0
v_ye = 0 
v_ze = 0 
w_xe = 0 
w_ye = 0 
w_ze = 0 
tau_xe = 0 
tau_ye = 0 
tau_ze = 0
f_ze = 9.81/2 


x = sym.Matrix([p_x, p_y, p_z, psi, theta, phi, v_x, v_y, v_z, w_x, w_y, w_z])
u = sym.Matrix([tau_x, tau_y, tau_z, f_z])




f_num = sym.lambdify((p_x, p_y, p_z, psi, theta, phi, v_x, v_y, v_z, w_x, w_y, w_z, tau_x, tau_y, tau_z, f_z), f)

A_num = sym.lambdify((p_x, p_y, p_z, psi, theta, phi, v_x, v_y, v_z, w_x, w_y, w_z, tau_x, tau_y, tau_z, f_z), f.jacobian(x))
A = A_num(p_xe, p_ye, p_ze, psie, thetae, phie, v_xe, v_ye, v_ze, w_xe, w_ye, w_ze, tau_xe, tau_ye, tau_ze, f_ze)

B_num = sym.lambdify((p_x, p_y, p_z, psi, theta, phi, v_x, v_y, v_z, w_x, w_y, w_z, tau_x, tau_y, tau_z, f_z), f.jacobian(u))
B = B_num(p_xe, p_ye, p_ze, psie, thetae, phie, v_xe, v_ye, v_ze, w_xe, w_ye, w_ze, tau_xe, tau_ye, tau_ze, f_ze)

g_num = sym.lambdify((p_x, p_y, p_z, psi, theta, phi, v_x, v_y, v_z, w_x, w_y, w_z, tau_x, tau_y, tau_z, f_z), g)

C_num = sym.lambdify((p_x, p_y, p_z, psi, theta, phi, v_x, v_y, v_z, w_x, w_y, w_z, tau_x, tau_y, tau_z, f_z), g.jacobian([p_x, p_y, p_z, psi, theta]))
C = C_num(p_xe, p_ye, p_ze, psie, thetae, phie, v_xe, v_ye, v_ze, w_xe, w_ye, w_ze, tau_xe, tau_ye, tau_ze, f_ze)

f_check = f_num(p_xe, p_ye, p_ze, psie, thetae, phie, v_xe, v_ye, v_ze, w_xe, w_ye, w_ze, tau_xe, tau_ye, tau_ze, f_ze)

g_check = g_num(p_xe, p_ye, p_ze, psie, thetae, phie, v_xe, v_ye, v_ze, w_xe, w_ye, w_ze, tau_xe, tau_ye, tau_ze, f_ze)
g_check
#sym.shape(A)



(12, 12)

In [17]:
from scipy import linalg
def lqr(A,B,Q,R):
    P=linalg.solve_continuous_are(A,B,Q,R)
    K=linalg.inv(R) @ B.T @ P
    return K

In [10]:
Wc = np.block([B, A@B, A@A@B, A@A@A@B, A@A@A@A@B, A@A@A@A@A@B])
cont = np.linalg.matrix_rank(Wc)


In [13]:
Qc = np.eye(12)
Rc = (np.eye(4))
K = lqr(A, B, Qc, Rc)

print(K) 
F = A-B@K
np.linalg.eigvals(F)


[[ 0.         -1.          0.         -0.         -0.          5.46213218
   0.         -1.45381723  0.          1.01248497  0.         -0.        ]
 [ 1.          0.         -0.         -0.          5.46213218 -0.
   1.45381723  0.         -0.          0.          1.01248497  0.        ]
 [ 0.          0.          0.          1.          0.         -0.
   0.          0.          0.         -0.          0.          1.00399203]
 [-0.         -0.          1.          0.         -0.          0.
  -0.         -0.          1.41421356  0.         -0.          0.        ]]


array([-434.78145927+0.j        , -434.78145927+0.j        ,
       -249.99799996+0.j        ,   -2.21469792+2.21475479j,
         -2.21469792-2.21475479j,   -2.21469792+2.21475479j,
         -2.21469792-2.21475479j,   -1.41421356+0.00000016j,
         -1.41421356-0.00000016j,   -1.00000003+0.j        ,
         -1.00000003+0.j        ,   -1.000008  +0.j        ])

In [40]:
Wo = np.block([C.T, A.T@C.T, (A.T)**2@C.T, (A.T)**3@C.T, (A.T)**4@C.T, (A.T)**5@C.T])
obsv = np.linalg.matrix_rank(Wo)


ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 5 is different from 12)

In [79]:
Qo = np.eye(12)
Ro = (np.eye(5))

L = lqr(A.T, C.T, np.linalg.inv(Ro), np.linalg.inv(Qo)).T


ValueError: Matrix a and b should have the same number of rows.